# 1- Importing the necessary libraries

In [1]:
import os as os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image,ImageOps
import pickle as pic
import math
scale = 0.4

# 2- Reading the parameters.txt

In [2]:
#reading the parameters from the parameters.txt file
sim_name = 'interlayer_1e-3_xi_alpha1_0j1.0_t1.5'
with open(sim_name + '/parameters.txt') as f:
    parameters = f.read()
print()
exec(parameters)

In [3]:
parameters

"stage_name='complete_1_0';outer='alpha';inner='xi';frame_range=[1,194];tile_size=[100,100];folder_name='interlayer_1e-3_xi_alpha1_0j1.0_t1.5';"

# 3- Specifiying the parameters to extract
## NOTE: If you choose the complete set of parameters to extract, in the saving step you may run out of ram and get a dead kernell. So just specify the necessary parameters. You can do it in the second next block. Do not touch the next block. Next block is only for reference

In [4]:
# Do not touch. Only for reference. specify the parameters in the next block
pars = ['Aex',
'alpha',
'anisC1',
'anisC2',
'anisU',
'B_anis',
'B_demag',
'B_eff',
'B_exch',
'B_ext',
'B_therm',
'Dbulk',
'Dind',
'Edens_anis',
'Edens_demag',
'Edens_exch',
'Edens_therm',
'Edens_total',
'Edens_Zeeman',
'EpsilonPrime',
'ExchCoupling',
'ext_topologicalchargedensity',
'FixedLayer',
'frozenspins',
'geom',
'J',
'Kc1',
'Kc2',
'Kc3',
'Ku1',
'Ku2',
'Lambda',
'LLtorque',
'm',
'm_full',
'MFM',
'Msat',
'regions',
'spinAngle',
'STTorque',
'torque'];

## Specify parameters in this block

In [5]:
pars = ['m']

# 4- Data preparation and function definition

In [6]:
os.chdir(sim_name)
stage_name += '.out'# This is the name of your simulations
outer_dirs = [x for x in os.listdir() if os.path.isdir(x) and '=' in x]
inner_dirs = [x for x in os.listdir(outer_dirs[0]) if
             os.path.isdir(outer_dirs[0] + '/' + x) and '=' in x]
f = lambda x: np.double(x.split('=')[1])
inner_dirs = sorted(inner_dirs , key = f)
outer_dirs = sorted(outer_dirs , key = f)
inner_values = [f(x) for x in inner_dirs]
outer_values = [f(x) for x in outer_dirs]
parameters_dict = {'stage_name':stage_name, 'outer':outer, 'inner':inner,
             'outer_values':outer_values, 'inner_values':inner_values}

In [7]:
def return_one_image():
    lists = os.walk('.')
    for a,b,c in lists:
        for item in c:
            if '.jpg' in item:
                return(a+'/' + item)

In [8]:
def single_image(par, frame_number, inner_dirs, outer_dirs, image_shape,flip=1,scale=1):
    l_inner = len(inner_dirs)
    l_outer = len(outer_dirs)
    Rs = image_shape[0]
    Cs = image_shape[1]
    big_image_dimensions = [Rs*(l_outer), Cs*l_inner , 3]
    big_image = np.uint8(np.zeros(big_image_dimensions))
    im_non = np.uint8(np.zeros(image_shape))
    image_indices = list()
    images = list()
    row = 0
    for outer_dir in outer_dirs:
        column = 0;
        exec(outer_dir)
        for inner_dir in inner_dirs:
            exec(inner_dir)
            try:
                img_name = outer_dir + '/' + inner_dir + '/' + stage_name + '/' + par +'%06d'%frame_number+'.jpg'
                im = Image.open(img_name)
                if flip ==1:
                    im = ImageOps.flip(im)
                im = np.array(im)
            except:
                im = im_non
            big_image[row*Rs:(row + 1)*Rs, 
                 column*Cs:(column + 1)*Cs, 
                 :] = im
            column +=1
        row += 1
    big_image = Image.fromarray(big_image)
    big_image = ImageOps.scale(big_image,scale)
    big_image = np.array(big_image)
    return big_image

In [9]:
sample_image= return_one_image() #extracting single image for reference
img = np.array(Image.open(sample_image))
image_shape = img.shape

# 5- Data extraction

In [10]:
total_frames = list()
for par in pars:
    print(par)
    frames = list()
    for frame_no in range(frame_range[0] , frame_range[1] + 1):
        one_frame = single_image(par, frame_no, inner_dirs, outer_dirs, image_shape,scale = scale)
        print(frame_no)# remove this line if the output is annoying. this prints out the extracted frame number
        frames.append(one_frame)
    total_frames.append(frames)

m
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194


In [11]:
output = {'data': dict(zip(pars , total_frames)) , 'parameters':parameters_dict}

# 6-saving

In [12]:
import gzip
file_name = folder_name +'.sims' #you can change this to whatever you want. but it's better if you don't change the extention i.e. '.sims'
with gzip.GzipFile(file_name, 'w') as f:
    pic.dump(output ,f)